# Cross Lingual summarization CNN Daily Mail Results
We will try out the trained t5 network from the tpu

In [1]:
import tensorflow as tf
import pandas as pd
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import time
from rouge_score import rouge_scorer
from rouge_score import scoring

In [2]:
if not tf.config.list_physical_devices('GPU'):
    print("Change runtime to \"GPU runtime\" for faster computations")

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Params

In [4]:
BATCH_SIZE = 8

SHUFFEL_SIZE = 1024

learning_rate = 3e-5

model_size = "t5-base"

MAX_ARTICLE_LEN = 512

MAX_HIGHLIGHT_LEN = 150

## Model

In [5]:
tokenizer = T5Tokenizer.from_pretrained(model_size)
model = TFT5ForConditionalGeneration.from_pretrained(model_size)

task_specific_params = model.config.task_specific_params
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))
    
pad_token_id = tokenizer.pad_token_id

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  24674304  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  84954240  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  113275008 
Total params: 222,903,552
Trainable params: 222,903,552
Non-trainable params: 0
_________________________________________________________________


In [7]:
ckpt_file = "../models/checkpoint_cross_lingual_sum.ckpt"
model.load_weights(ckpt_file)

## Dataset
We will load the translated CNN Daily Mail dataset from the tfrecords files

In [8]:
en_de_prefix = tf.reshape(tokenizer.encode("summarize: en_to_ger ", return_tensors="tf"), (-1,))
de_en_prefix = tf.reshape(tokenizer.encode("summarize: ger_to_en ", return_tensors="tf"), (-1,))
en_en_prefix = tf.reshape(tokenizer.encode("summarize: en_to_en ", return_tensors="tf"), (-1,))
de_de_prefix = tf.reshape(tokenizer.encode("summarize: ger_to_ger ", return_tensors="tf"), (-1,))

In [9]:
prefix_length = de_de_prefix.shape[0]
prefix_length

9

In [45]:
import numpy as np
MAX_ARTICLE_LEN = 512
MAX_HIGHLIGHT_LEN = 150
GLOBAL_BATCH_SIZE = 8

def get_tfrecord_dataset(file_name):
    features = {
        'ger_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'ger_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'ger_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'ger_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),

        'en_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'en_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'en_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'en_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
    }

    dataset = tf.data.TFRecordDataset(f"../data/{file_name}.tfrecord")

    # Taken from the TensorFlow models repository: https://github.com/tensorflow/models/blob/befbe0f9fe02d6bc1efb1c462689d069dae23af1/official/nlp/bert/input_pipeline.py#L24
    def decode_record(record, features):
        """Decodes a record to a TensorFlow example."""
        example = tf.io.parse_single_example(record, features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.cast(t, tf.int32)
            example[name] = t
        return example


    def select_data_from_record(record):
        return [
            tf.strided_slice(tf.concat([de_de_prefix, record['ger_x']], axis=0), [0], [-1]), tf.concat([tf.ones(8, dtype=tf.int32), record['ger_x_mask']], axis=0), record['ger_y'], record['ger_y_ids'],
            tf.strided_slice(tf.concat([en_de_prefix, record['en_x']], axis=0), [0], [-1]), tf.concat([tf.ones(8, dtype=tf.int32), record['en_x_mask']], axis=0), record['ger_y'], record['ger_y_ids'],
            tf.strided_slice(tf.concat([de_en_prefix, record['ger_x']], axis=0), [0], [-1]), tf.concat([tf.ones(8, dtype=tf.int32), record['ger_x_mask']], axis=0), record['en_y'], record['en_y_ids'],
            tf.strided_slice(tf.concat([en_en_prefix, record['en_x']], axis=0), [0], [-1]), tf.concat([tf.ones(8, dtype=tf.int32), record['en_x_mask']], axis=0), record['en_y'], record['en_y_ids'],
        ]
    
    dataset = dataset.map(lambda record: decode_record(record, features))
    dataset = dataset.map(select_data_from_record)
    dataset = dataset.shuffle(25000)
    return dataset.batch(GLOBAL_BATCH_SIZE)

test_ds = get_tfrecord_dataset("corss_lingual_test_cnn_daily_mail")

In [46]:
def get_summaries(ds):
    for i in range(1,5):
        yield ds[(i-1)*4], ds[i*4-3], ds[i*4-2], ds[i*4-1]


for ds in test_ds.take(1):
    for i in get_summaries(ds):
        print(i[0].shape, i[1].shape, i[2].shape, i[3].shape)

(8, 512) (8, 512) (8, 150) (8, 150)
(8, 512) (8, 512) (8, 150) (8, 150)
(8, 512) (8, 512) (8, 150) (8, 150)
(8, 512) (8, 512) (8, 150) (8, 150)


## Evaluation
### Define Rouge Score

In [47]:
class RougeScore:
    '''
    mostly from https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py 
    '''
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            # Make sure the summary is not bytes-type
            # Add newlines between sentences so that rougeLsum is computed correctly.
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

### Compute Summaries

In [48]:
predictions = []
start_time = time.time()

for i, ds_item in enumerate(test_ds): 
    for (input_ids, input_mask, y, y_ids) in get_summaries(ds_item):
        summaries = model.generate(
            input_ids=input_ids, 
            attention_mask=input_mask, 
            num_beams=4, 
            length_penalty=0.6,
            early_stopping=True, 
            max_length=150
        )

        articles = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in input_ids]

        pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
        real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
    
        for pred_sent, real_sent, article_sent in zip(pred, real, articles):
            predictions.append(str("article: " + article_sent + "\n\npred sentence: " + pred_sent + "\n\nreal sentence: " + real_sent))
    
    if (i % 10) == 0:
        elapsed = (time.time() - start_time) / 10
        print(i,": time genreate batch:", elapsed)
        start_time = time.time()
    if i > 20:
        # otherwise it will take ages
        break


# rouge_score.result()

0 : time genreate batch: 15.100758624076843
10 : time genreate batch: 169.7196108341217
20 : time genreate batch: 171.28110806941987


### Lets have a look at some of these predicted summaries

In [49]:
import numpy as np
len_predictions = len(predictions)

def get_random_prediction():
    return predictions[np.random.randint(len_predictions)]

In [50]:
print(get_random_prediction())

article: summarize: ger_to_en Thierry Henry hat mit einem 84-minütigen Video von Michael Laudrups Tod auf Kritiker reagiert, die mit seinen Ansichten über Javier Hernandez "Feier nicht einverstanden waren. Der ehemalige Arsenal-Stürmer Henry, heute Experte bei Sky Sports, nahm Hernandez, der nach seinem Siegtor für Real Madrid am Mittwochabend gefeiert hatte, "als hätte er die Weltmeisterschaft gewonnen", in Schutz und sagte nach dem Spiel, er schulde seinem Teamkollegen Cristiano Ronaldo alles. Der dreimalige Weltfußballer des Jahres hätte im Champions-League-Viertelfinal-Rückspiel von Real Madrid gegen den erbitterten Rivalen Atlético zwei Minuten vor Schluss einschießen können, entschied sich aber stattdessen für Teamkollege Hernandez, der den Ball selbst zum Sieg versenkte. Real Madrids Stürmer Javier Hernandez verdiente sich den Champions-League-Sieg mit einem späten Tor. Der mexikanische Leihstürmer ließ sich nach seinem Führungstreffer von Madrid feiern. Hernandez wird von den v

In [51]:
print(get_random_prediction())

article: summarize: ger_to_ger Vergessen Sie ein Daunenkissen - dieser Junge würde lieber den Bauch seiner Katze als Ruheplatz nutzen. Der kalifornische YouTube-Nutzer Lionel Hutz filmte, wie sein kleiner Sohn die Katze der Familie durch den Garten jagte, um sich auszuruhen. Filmaufnahmen zeigen, wie die Katze ihr Fell leckt, bevor sein menschlicher Freund daherkommt und sich obenauf plappert. Nach einem kurzen Moment der Stille steht die Katze sofort auf und huscht an eine andere Stelle. Anschließend folgt der Junge dem Haustier zu seinem neuen Standort. Eifrig, sich auszuruhen, wiederholt der Säugling dieselbe Bewegung und rollt zu Boden. Die Sequenz passiert ein weiteres Mal, bevor die Katze beschließt, dass sie genug davon hat, als Kissen benutzt zu werden und im Haus umherirrt. Auf einem späteren Video ist zu sehen, wie sich das Duo schnell schminkt und der Junge sanft die Katzenhände mit Leckereien füttert. Schläfriger Kopf: YouTube-User Lionel Hutz filmte, wie sein kleiner Sohn 

In [52]:
print(get_random_prediction())

article: summarize: ger_to_ger Es ist eine weit verbreitete Angst, die als Trypanophobie bekannt ist. Bei rund 20 Prozent der Bevölkerung ruft der Gedanke, zur Spritze zum Arzt zu gehen, ein tief verwurzeltes Gefühl der Angst hervor. Viele meiden die Arztpraxis ganz und lassen sich das Risiko verschiedener Krankheiten und Infektionen offen. Doch für diejenigen, die Angst vor Nadeln haben, könnte ein Wissenschaftlerteam der Rice University in Houston die Antwort parat haben. Sie haben ein neues Gerät entwickelt, Comfortably Numb, um die Schmerzen einer Injektion zu lindern. Ihre Erfindung betäubt die Haut vor dem Jab, indem sie eine schnelle chemische Reaktion hervorruft, um die Haut des Patienten zu kühlen. Scrollen Sie nach unten für ein Video. Comfortably Numb ist ein neues Einmalgerät, das die Schmerzen einer Injektion lindert, indem es eine schnelle chemische Reaktion erzeugt, um die Haut des Patienten innerhalb von 60 Sekunden zu kühlen. Drei Studenten im ersten Jahr, (von links) 

In [53]:
print(get_random_prediction())

article: summarize: ger_to_ger Im TOWIE am Sonntagabend erklärte sich Dan Edgar bereit, seiner Beziehung zu Jessica Wright eine zweite Chance zu geben - und angesichts der Beweise für ihr neuestes Fotoshooting überrascht es nicht. Die 29-Jährige wurde als Gesicht (und Körper) von Ann Summers Bademoden- und Beachwear-Kampagne Hotel Summers angekündigt. Der TOWIE-Star ist der erste Promi, der jemals vor einer Ann Summers-Kampagne stand und wurde nicht nur aufgrund ihrer atemberaubenden Figur ausgewählt, sondern auch wegen ihrer "lustigen, furchtlosen Haltung und Referenzen im Modegeschäft". Jessica Wright von TOWIE ist die erste Berühmtheit, die in einer Kampagne von Ann Summers mitspielt. In der Kampagne präsentiert Jess ihre unglaubliche Figur in der Resort-Kollektion, die 15 Bademoden umfasst, darunter Tauch-, Bandeau- und Boost-Bikinis sowie kurvenverstärkende Monokinis. Neben der Bademode umfasst die Kollektion Bademoden-Accessoires, darunter einen bestickten Kaftan, ein florales Co

In [54]:
print(get_random_prediction())

article: summarize: ger_to_ger Die Jacksonville Jaguars werden im nächsten Monat einen kleinen Teil des NFL-Drafts nach London bringen, indem sie von einem in Großbritannien ansässigen Fan live im Fernsehen ihre Pick-ups der sechsten und siebten Runde verkünden lassen. Die Jaguars, die bis 2015 jedes Jahr ein Heimspiel in London austragen wollen, haben über nfluk.com einen Wettbewerb für Fans aus Großbritannien eröffnet, und der Gewinner wird die Draft Picks live auf Sky Sports in Großbritannien und dem NFL Network in den USA bekanntgeben. Es wird das erste Mal sein, dass ein Teil des NFL-Drafts außerhalb der Vereinigten Staaten stattfindet. Ein in Großbritannien ansässiger Fan wird die sechsten und siebten Draft-Picks der Jacksonville Jaguars live im Fernsehen verkünden. Der britische NFL-Fan könnte der Welt die Antwort auf 2014 No 3 Pick Blake Bortles verkünden. Jaguars-Präsident Mark Lamping sagte: "Wir sind so aufgeregt, unseren Fans in den USA und im Ausland eine aufregende Gelege

In [55]:
for i in range(10):
    print(get_random_prediction())

article: summarize: en_to_en This is the moment a thief fled the scene after stealing the sat nav from a 73-year-old injured pensioners crashed car as she waited for an ambulance. The 73-year-old woman suffered severe bruising to her legs and ribs after her blue Vauxhall Corsa collided with railings on the petrol forecourt of the Asda store in Hulme, Manchester. She was waiting from an ambulance with her husband, who has dementia, when the thief sneaked into her car and stole the black Garmin device from underneath the passenger seat. A member of the public took a photograph of the suspect as he left the scene and walked across the grass with his dog at around 12pm last Thursday. A man wearing purple shorts was photographed leaving the scene after a sat nav was stolen from a 73-year-old injured pensioner's crashed car in Hulme, Manchester . The man is described as being between 30 and 40, with a beard and wearing purple surf-style shorts. Police have issued the photograph of the suspec

## Save results to text file

In [66]:
result_path = "../results/t5_cross_lingual_plus_translate_sum_result.txt"
open(result_path, "w")
for pred in predictions:
    with open(result_path, "a") as file:
        file.write(pred + "\n")

## Load save File

In [67]:
data_points = []
result_path = "../results/t5_cross_lingual_plus_translate_sum_result.txt"
file = open(result_path, "r")
for line in file:
    data_points.append(line)

In [68]:
data_points[4]

'real sentence: Manager des London Ambulance Service besetzen gerade 225 unbesetzte Stellen. Bewerber alle aus Sydney und Melbourne rekrutiert nach einem Tag der Tests. Jüngstes Beispiel dafür, wie NHS Personal im Ausland rekrutiert, weil es in Großbritannien nicht ausgebildet wird. Krankenhausmanager stellen Chargen von 30 Krankenschwestern ein, die auf Reisen nach Spanien sind. Sanitätern werden 4.500 Pfund goldene Hellos angeboten, wenn sie innerhalb von drei Monaten nach Großbritannien ziehen. Die Manager sind so bestrebt, Stellen zu besetzen, dass 91 Prozent derjenigen, die zu Bewertungen erschienen sind,\n'

In [69]:
class SummaryData():
    
    def __init__(self):
        self.language_tag = ''
        self.real_data = ''
        self.pred_data = ''        

In [70]:
count = 0
summary_data = SummaryData()
summary_data_list = []
for point in data_points:
    count += 1
    
    if count == 1:
        summary_data.language_tag = point.split(" ")[2]
    elif count == 3:
        summary_data.pred_data = ": ".join(point.split(":")[1:])
    elif count == 5:
        summary_data.real_data = ": ".join(point.split(":")[1:])
        summary_data_list.append(summary_data)
        summary_data = SummaryData()
        count = 0
summary_data_list[0].__dict__

{'language_tag': 'ger_to_ger',
 'real_data': ' Manager des London Ambulance Service besetzen gerade 225 unbesetzte Stellen. Bewerber alle aus Sydney und Melbourne rekrutiert nach einem Tag der Tests. Jüngstes Beispiel dafür, wie NHS Personal im Ausland rekrutiert, weil es in Großbritannien nicht ausgebildet wird. Krankenhausmanager stellen Chargen von 30 Krankenschwestern ein, die auf Reisen nach Spanien sind. Sanitätern werden 4.500 Pfund goldene Hellos angeboten, wenn sie innerhalb von drei Monaten nach Großbritannien ziehen. Die Manager sind so bestrebt, Stellen zu besetzen, dass 91 Prozent derjenigen, die zu Bewertungen erschienen sind,\n',
 'pred_data': ' Die Rekrutierung erfolgte auf der Grundlage einer Reihe praktischer Vorführungen an Attrappen, einer schriftlichen Prüfung und eines Interviews, das insgesamt zwischen vier und sechs Stunden dauerte. Es ist das jüngste Beispiel dafür, wie der NHS im Ausland rekrutiert, weil er es versäumt hat, ausreichend Personal in Großbritanni

In [71]:
rouge_scores_dict = dict()
rouge_scores_dict['en_to_en'] = RougeScore()
# rouge_scores_dict['en_to_en_trans'] = RougeScore()

rouge_scores_dict['en_to_ger'] = RougeScore()
rouge_scores_dict['ger_to_en'] = RougeScore()
rouge_scores_dict['ger_to_ger'] = RougeScore()
# rouge_scores_dict['ger_to_ger_trans'] = RougeScore()

rouge_scores_dict

{'en_to_en': <__main__.RougeScore at 0x7ff6cc255748>,
 'en_to_ger': <__main__.RougeScore at 0x7ff6cc255fd0>,
 'ger_to_en': <__main__.RougeScore at 0x7ff6cc255c18>,
 'ger_to_ger': <__main__.RougeScore at 0x7ff6cc2553c8>}

In [72]:
for summary_data in summary_data_list:
    rouge_scores_dict[summary_data.language_tag](summary_data.real_data, summary_data.pred_data)
    

In [73]:
results_ger_trans[0]

{'id': 24,
 'item_id': 1,
 'language_tag': 'en_to_en',
 'real_data': 'Beatrice seen watching race on terrace with the Gulf states Crown Prince . Marks 13th holiday since November last year, and fourth in a month . Princess quit her job at Sony Pictures in New York before Christmas . Despite that she is described as working full-time on her fathers website .\n',
 'pred_data': 'Princess Beatrice spotted at Bahrain Grand Prix with long-term boyfriend Dave Clark . Onlooker said 26-year-old was walking behind the Crown Prince of Bahrain . Princes regime accused of violently repressing pro-democracy protests . Sir Jackie Stewart and comedian Rory Bremner watched race from the tower .\n',
 'translated': 'Prinzessin Beatrice beim Großen Preis von Bahrain mit ihrem langjährigen Freund Dave Clark.Beobachter sagten, der 26-Jährige sei hinter dem Kronprinzen von Bahrain hergelaufen.Prinzen-Regime beschuldigt, prodemokratische Proteste gewaltsam zu unterdrücken.Sir Jackie Stewart und Komiker Rory B

In [74]:
for key, rouge_score_item in rouge_scores_dict.items():
    print(key, rouge_score_item.result())
    print()

rouge1 = 40.15, 95% confidence [37.90, 42.35]
rouge2 = 19.72, 95% confidence [17.71, 21.96]
rougeLsum = 37.34, 95% confidence [35.21, 39.48]
en_to_en {'rouge1': 40.152332466743346, 'rouge2': 19.72039258791332, 'rougeLsum': 37.34013650154572}

rouge1 = 33.56, 95% confidence [31.66, 35.56]
rouge2 = 13.92, 95% confidence [12.26, 15.46]
rougeLsum = 24.33, 95% confidence [22.74, 26.03]
en_to_ger {'rouge1': 33.56388151677695, 'rouge2': 13.91534537250046, 'rougeLsum': 24.332827792341384}

rouge1 = 36.43, 95% confidence [34.66, 38.18]
rouge2 = 14.53, 95% confidence [13.16, 15.88]
rougeLsum = 33.73, 95% confidence [32.05, 35.46]
ger_to_en {'rouge1': 36.42991955605362, 'rouge2': 14.531216544033743, 'rougeLsum': 33.72602515298565}

rouge1 = 33.30, 95% confidence [31.14, 35.54]
rouge2 = 14.80, 95% confidence [12.76, 17.14]
rougeLsum = 24.27, 95% confidence [22.21, 26.61]
ger_to_ger {'rouge1': 33.299454057900256, 'rouge2': 14.80379056363354, 'rougeLsum': 24.270727077520487}



# Special Example

In [29]:
input_text = "In a world where we have to read and understand a lot of documents automatic text summarization has an obvious demand. To have the option to get a brief summary of a text in your language can be very useful. But would it not be even more useful, if you could have the option to get a summary of a text in a language, you do not understand, in your wished language? Sometimes we don’t want an exact translation, sometimes we just want to know a brief overview of a text in a language we don’t understand. That is the case where Cross-Lingual summarization would be prefered over normal translation. Cross-Lingual summarization can give you a short overview of a text in a language you do not understand."

In [30]:
input_text_test = "The dramatic growth of data on the internet leads to the need to automatically process and understand the data. A big part of the data is text data in many languages. This overwhelming amount of information causes a demand for automatic text summarization and other Natural Language Processing (NLP) taks.\n The Field of NLP had some high points in the last couple of years, when the field got revolutionized by Neural Language Models. With publications like Attention is all you need \cite{vaswani2017attention} or GPT3\cite{brown2020language} the limits of the field are pushed even further. Because of large data corpuses, scraped from the internet, and advanced models which contain up to 175 Billion parameters like the GPT3 it is possible to generate text, answer questions, summarize text, translate or many other things.\n In this master thesis we will take a deeper look into summarization. There are extractive and abstractive techniques to summarization. The extractive summary technique tries to find subsets of sentences, which representante the original text well and uses them to summarize the original text \cite{allahyari2017text}. The abstractive technique uses advanced language models to generate a new text, which should be much shorter than the original one, contain all the key information and preserve the overall meaning.\n In this master thesis we will use the abstractive technique, because it is closer to a human-like interpretation. It combines the ability to understand what the context of a given text is and the ability to generate fluent and grammatically correct text to that given context.\n The summarization in one language is an interesting topic but cross lingual approach is even more interesting. The goal of Cross Lingual Summarization is to summarize a text from one language into another language. It combines the ability to summarize and the ability to translate. Where state of the art models perform well on normal summarization, it will be interesting to see how well they perform doing Cross-Lingual Summarization."

In [31]:
x = tokenizer.encode_plus("summarize: en_to_ger " + input_text, max_length=512, return_tensors="tf", padding='max_length', truncation=True)
input_ids = tf.reshape(x['input_ids'], (1,-1))
attention_mask = tf.reshape(x['attention_mask'], (1,-1))

In [32]:
print(input_ids.shape, attention_mask.shape)

(1, 512) (1, 512)


In [33]:
summaries = model.generate( 
    input_ids=input_ids, 
    attention_mask=attention_mask
)

In [34]:
summaries

<tf.Tensor: shape=(1, 74), dtype=int32, numpy=
array([[    0,    86,   645,  3779,     6,    16,    74,   558,  2584,
            3, 20127,    15,   110,    35,    64, 19163,  3766,     6,
          229, 15820,    15,  5027,     7,    63,    29, 19712,   266,
        30410,    15,  5222,  6367,     5,  4098,  6199,     3,    15,
            7,   311, 30001,    49,     6,  1301,   292,    67,  7251,
         8219,    29,     6,   266, 11068, 14449,   266,     7,  5027,
           15,     7,    16,   645, 16933,   170,  8837,     6,    67,
          292,   311, 19163,     6,    16,  1197,    52, 24054,    29,
        16933,    58]], dtype=int32)>

In [35]:
tokenizer.decode(summaries[0])

'<pad> In einer Welt, in der wir viele Dokumente lesen und verstehen müssen, ist automatische Textsynthese eine offensichtliche Forderung. Aber wäre es nicht nützlicher, wenn Sie die Möglichkeit hätten, eine Zusammenfassung eines Textes in einer Sprache zu bekommen, die Sie nicht verstehen, in Ihrer gewünschten Sprache?'